In [1]:
import numpy as onp
#import jax.numpy as np
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rc
from scipy.interpolate import griddata

from scipy import stats

plt.rcParams.update(plt.rcParamsDefault)

plt.rc('font', family='serif')
plt.rcParams.update({'font.size': 11,
                     #'lines.linewidth': 2,
                     #'axes.labelsize': 20,  # fontsize for x and y labels (was 10)
                     #'axes.titlesize': 20,
                     #'xtick.labelsize': 16,
                     #'ytick.labelsize': 16,
                     #'legend.fontsize': 20,
                     #'axes.linewidth': 2,
                     "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
                     "text.usetex": True,                # use LaTeX to write all text
                     })




In [2]:
regrets_LCB = np.load("regrets_LCB.npy")
regrets_LCB_v = np.load("regrets_LCB_v.npy")
regrets_mean = np.load("regrets_mean.npy")
regrets_TS = np.load("regrets_TS.npy")
regrets_EI = np.load("regrets_EI.npy")
regrets_LW_LCB = np.load("regrets_LW_LCB.npy")

X_LCB = np.load("X_LCB.npy")
X_LCB_v = np.load("X_LCB_v.npy")
X_mean = np.load("X_mean.npy")
X_TS = np.load("X_TS.npy")
X_EI = np.load("X_EI.npy")
X_LW_LCB = np.load("X_LW_LCB.npy")

Weights_LW_LCB = np.load("Weights_LW_LCB.npy")


T = regrets_LCB.shape[0]

def wheel_function(x, y, delta_control):
    # define the control parameter
    reward0 = 0.2
    reward1 = 1.0
    reward2 = 0.1
    reward3 = 0.05
    reward4 = 0.0
    
    # the number of modes in this direction
    r = np.sqrt(x**2 + y**2)
    if r < delta_control:
        return reward0
    elif (x >= 0) & (y >= 0):
        return reward1
    elif (x >= 0) & (y <= 0):
        return reward2
    elif (x <= 0) & (y >= 0):
        return reward3
    else:
        return reward4
    
MC_samples = 100
nn = 60
delta_control = 0.7


# Locations for bandits
xx = onp.linspace(-1, 1, nn)
yy = onp.linspace(-1, 1, nn)
XX, YY = onp.meshgrid(xx, yy)
X_positions = onp.concatenate([XX.flatten()[:,None], YY.flatten()[:,None]], axis = 1)

# find the valid locations
R = onp.sqrt(X_positions[:,0]**2 + X_positions[:,1]**2)
X_positions = X_positions[R <= 1]

# Draw training data from functions
Y_data = onp.ones((X_positions.shape[0], 1))
for k in range(X_positions.shape[0]):
    Y_data[k, 0] = wheel_function(X_positions[k,0], X_positions[k,1], delta_control)


# Normalize the data
#Y_data = (Y_data - np.mean(Y_data)) / np.std(Y_data)

Y_data = onp.tile(Y_data,(1, MC_samples))
Y_data = -Y_data
print(Y_data.shape)


xs = X_positions

sigma = np.sqrt(1e-6) 
T = 100

# Plot the first snapshot as the data
Y_data_plot = griddata((X_positions[:,0], X_positions[:,1]), Y_data[:,0], (XX, YY), method='linear')


times = np.arange(T) + 1



regrets_LCB = np.cumsum(regrets_LCB, axis = 1) / times
regrets_LCB_v = np.cumsum(regrets_LCB_v, axis = 1) / times
regrets_mean = np.cumsum(regrets_mean, axis = 1) / times
regrets_TS = np.cumsum(regrets_TS, axis = 1) / times
regrets_EI = np.cumsum(regrets_EI, axis = 1) / times
regrets_LW_LCB = np.cumsum(regrets_LW_LCB, axis = 1) / times



dispersion_scale = 0.2

y1_mu, y1_std = np.median(regrets_LCB, axis = 0), stats.median_abs_deviation(regrets_LCB, axis = 0)
y2_mu, y2_std = np.median(regrets_mean, axis = 0), stats.median_abs_deviation(regrets_mean, axis = 0)
y4_mu, y4_std = np.median(regrets_TS, axis = 0), stats.median_abs_deviation(regrets_TS, axis = 0)
y5_mu, y5_std = np.median(regrets_EI, axis = 0), stats.median_abs_deviation(regrets_EI, axis = 0)
y6_mu, y6_std = np.median(regrets_LW_LCB, axis = 0), stats.median_abs_deviation(regrets_LW_LCB, axis = 0)
y7_mu, y7_std = np.median(regrets_LCB_v, axis = 0), stats.median_abs_deviation(regrets_LCB_v, axis = 0)





(2724, 100)


In [3]:
plt.close("all")
plt.figure(figsize=(12*0.85,2.8*0.85))

ax = plt.subplot(141)

#plt.contourf(XX, YY, -Y_data_plot, 20, cmap="viridis")
plt.imshow(-Y_data_plot, extent=[-1, 1, -1, 1], origin='lower', cmap='plasma')
plt.plot(X_positions[:,0], X_positions[:,1], 'o', markersize=1,
             markerfacecolor='white', markeredgecolor="k",
             markeredgewidth=0.2, linestyle='None')
#plt.scatter(X_positions[:,0], X_positions[:,1], s=2, c=-Y_data_plot)

         #'g.', ms = 1, alpha = 0.5)
#plt.colorbar().ax.tick_params(labelsize=9)
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.gca().set_aspect('equal', adjustable='box')
ax.tick_params(direction='in', length=2)
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.xticks([-1,0,1])
plt.yticks([-1,0,1])
ax.set_xticklabels(["$-1$", "0", "1"])
ax.set_yticklabels(["$-1$", "0", "1"])
#ax.set_xticklabels(["0", r"$\pi/2$", r"$\pi$"])
#ax.set_yticklabels(["0", r"$\pi/2$", r"$\pi$"])
#plt.savefig('./Bandits.png', dpi = 200)

ax = plt.subplot(142)
lower1 = np.log10(np.clip(y1_mu - dispersion_scale*y1_std, a_min=0., a_max = np.inf) + 1e-8)
upper1 = np.log10(y1_mu + dispersion_scale*y1_std + 1e-8)
plt.fill_between(times.flatten(), lower1.flatten(), upper1.flatten(), 
                 facecolor='C4', alpha=0.3) #, label="Two std band")
#lower2 = np.log10(np.clip(y2_mu - dispersion_scale*y2_std, a_min=0., a_max = np.inf) + 1e-8)
#upper2 = np.log10(y2_mu + dispersion_scale*y2_std + 1e-8)
#plt.fill_between(times.flatten(), lower2.flatten(), upper2.flatten(), 
#                 facecolor='green', alpha=0.5) #, label="Two std band")
lower4 = np.log10(np.clip(y4_mu - dispersion_scale*y4_std, a_min=0., a_max = np.inf) + 1e-8)
upper4 = np.log10(y4_mu + dispersion_scale*y4_std + 1e-8)
plt.fill_between(times.flatten(), lower4.flatten(), upper4.flatten(), 
                 facecolor='C1', alpha=0.3) #, label="Two std band")
lower5 = np.log10(np.clip(y5_mu - dispersion_scale*y5_std, a_min=0., a_max = np.inf) + 1e-8)
upper5 = np.log10(y5_mu + dispersion_scale*y5_std + 1e-8)
plt.fill_between(times.flatten(), lower5.flatten(), upper5.flatten(), 
                 facecolor='C0', alpha=0.3) #, label="Two std band")
lower6 = np.log10(np.clip(y6_mu - dispersion_scale*y6_std, a_min=0., a_max = np.inf) + 1e-8)
upper6 = np.log10(y6_mu + dispersion_scale*y6_std + 1e-8)
plt.fill_between(times.flatten(), lower6.flatten(), upper6.flatten(), 
                 facecolor='C3', alpha=0.3) #, label="Two std band")
lower7 = np.log10(np.clip(y7_mu - dispersion_scale*y7_std, a_min=0., a_max = np.inf) + 1e-8)
upper7 = np.log10(y7_mu + dispersion_scale*y7_std + 1e-8)
plt.fill_between(times.flatten(), lower7.flatten(), upper7.flatten(), 
                 facecolor='C2', alpha=0.3) #, label="Two std band")



#plt.plot(times,np.log10(y2_mu),'g-', label = "median of mean", linewidth=2)
plt.plot(times,np.log10(y5_mu),'C0-', label = "EI", linewidth=1)
plt.plot(times,np.log10(y4_mu),'C1-', label = "TS", linewidth=1)
plt.plot(times,np.log10(y7_mu),'C2', label = "V-UCB", linewidth=1)
plt.plot(times,np.log10(y1_mu),'C4', label = "GP-UCB", linewidth=1)
plt.plot(times,np.log10(y6_mu),'C3-', label = "LW-UCB", linewidth=1)



plt.xlim(0,100)
plt.ylim(-0.65, -0.05)
plt.xlabel(r'Round $t$')
plt.ylabel(r'$\log\,R_t/t$')
ax.yaxis.set_label_coords(-0.23, 0.5)
plt.yticks([-0.65, -0.35, -0.05])
ax.tick_params(direction='in', length=2)
#plt.legend(loc='upper right', frameon=False, fontsize=9, labelspacing=0.2)#, borderaxespad=0.1)
plt.legend(loc='upper right', frameon=False, fontsize=9, labelspacing=0.2, borderaxespad=0.1, 
           handlelength=1.5, handletextpad=0.5)



ax = plt.subplot(143)
k = 18
weights_plot = Weights_LW_LCB[k,:,:]
X = X_LW_LCB[k,:,:]
X = np.unique(X, axis=0)

plt.contourf(XX, YY, weights_plot, cmap="Reds")
#plt.imshow(weights_plot, extent=[0, 1, 0, 1], origin='lower', cmap='Reds')
plt.plot(X[:,0], X[:,1], 'o', markersize=2,
             markerfacecolor='none', markeredgecolor="k",
             markeredgewidth=0.5, linestyle='None')
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.gca().set_aspect('equal', adjustable='box')
ax.tick_params(direction='in', length=2)
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.xticks([-1,0,1])
plt.yticks([-1,0,1])
ax.set_xticklabels(["$-1$", "0", "1"])
ax.set_yticklabels(["$-1$", "0", "1"])
plt.gca().set_aspect('equal', adjustable='box')
ax.tick_params(direction='in', length=2)
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')


ax = plt.subplot(144)
k = 69
weights_plot = Weights_LW_LCB[k,:,:]
X = X_LW_LCB[k,:,:]
X = np.unique(X, axis=0)

plt.contourf(XX, YY, weights_plot, cmap="Reds")
#plt.imshow(weights_plot, extent=[0, 1, 0, 1], origin='lower', cmap='Reds')
plt.plot(X[:,0], X[:,1], 'o', markersize=2,
             markerfacecolor='none', markeredgecolor="k",
             markeredgewidth=0.5, linestyle='None')
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.gca().set_aspect('equal', adjustable='box')
ax.tick_params(direction='in', length=2)
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.xticks([-1,0,1])
plt.yticks([-1,0,1])
ax.set_xticklabels(["$-1$", "0", "1"])
ax.set_yticklabels(["$-1$", "0", "1"])
plt.gca().set_aspect('equal', adjustable='box')
ax.tick_params(direction='in', length=2)
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')

#plt.tight_layout(w_pad=0)
plt.subplots_adjust(left=0.04, bottom=0.17, right=1, top=0.95, wspace=0.35, hspace=0)
plt.savefig('wheel7.pdf')
#plt.show()